## Functions for plotting metrics for comparison (text in Polish)

> Applies to local experiments not described in the code or documentation.

### Loss plot

In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

def plot_loss():
    sns.set_style("whitegrid")
    sns.set_context("paper", font_scale=1.2)
    plt.rcParams.update({
        "figure.figsize": (8,6),
        "figure.dpi": 300,
        "axes.linewidth": 1.0,
        "grid.linestyle": "--",
        "grid.alpha": 0.7,
        "legend.frameon": False,
        "legend.loc": "best"
    })

    research_dir = "..\\outputs\\research\\hqnn_parallel\\qubits"
    research_pl = "Liczba kubitów"
    metric = "loss"
    metric_pl = "Koszt"
    stage = "val"


    pattern = os.path.join(research_dir, f"[1-9]_{stage}_*.csv")
    file_paths = sorted(glob.glob(pattern), key=lambda x: int(os.path.basename(x).split("_",1)[0]))
    if not file_paths:
        raise Exception("Where?")

    data = {fp: pd.read_csv(fp) for fp in file_paths}

    fig, ax = plt.subplots()
    for fp, df in data.items():
        label = f"{research_pl}: {os.path.basename(fp).split('_',1)[0]}"
        ax.plot(df["epoch"], df[metric], label=label, linewidth=1.5, marker="o", markersize=3)
    ax.set_xlabel("Liczba epok")

    ax.set_ylabel(metric_pl)
    ax.tick_params(pad=5)
    ax.legend()

    axins = inset_axes(ax, width="30%", height="30%", loc="center", borderpad=2)
    axins.patch.set_facecolor("white")
    axins.set_zorder(2)
    for fp, df in data.items():
        x = df["epoch"].values[-5:]
        y = df[metric].values[-5:]
        axins.plot(x, y, linewidth=1.5, marker="o", markersize=3)

    x0 = min(df["epoch"].values[-5:].min() for df in data.values())
    x1 = max(df["epoch"].values[-5:].max() for df in data.values())
    y0 = min(df[metric].values[-5:].min() for df in data.values()) - 0.001
    y1 = max(df[metric].values[-5:].max() for df in data.values()) + 0.001

    axins.set_xlim(x0, x1)
    axins.set_ylim(y0, y1)
    axins.tick_params(labelsize=8)

    mark_inset(ax, axins, loc1=3, loc2=1, fc="none", ec="0.5", linestyle="--", linewidth=0.5, zorder=1)
    plt.tight_layout()
    fig.savefig(os.path.join(research_dir, f"{metric}_{stage}_comparison.png"))
    
plot_loss()

### Accuracy plot

In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

def plot_loss():
    sns.set_style("whitegrid")
    sns.set_context("paper", font_scale=1.2)
    plt.rcParams.update({
        "figure.figsize": (8,6),
        "figure.dpi": 300,
        "axes.linewidth": 1.0,
        "grid.linestyle": "--",
        "grid.alpha": 0.7,
        "legend.frameon": False,
        "legend.loc": "best"
    })

    research_dir = "..\\outputs\\research\\hqnn_parallel\\qubits"
    research_pl = "Liczba kubitów"
    metric = "accuracy"
    metric_pl = "Dokładność [%]"
    stage = "train"


    pattern = os.path.join(research_dir, f"[1-9]_{stage}_*.csv")
    file_paths = sorted(glob.glob(pattern), key=lambda x: int(os.path.basename(x).split("_",1)[0]))
    if not file_paths:
        raise Exception("Where?")

    data = {fp: pd.read_csv(fp) for fp in file_paths}

    fig, ax = plt.subplots()
    for fp, df in data.items():
        df[metric] *= 100.0 # %
        label = f"{research_pl}: {os.path.basename(fp).split('_',1)[0]}"
        ax.plot(df["epoch"], df[metric], label=label, linewidth=1.5, marker="o", markersize=3)
    ax.set_xlabel("Liczba epok")

    ax.set_ylabel(metric_pl)
    ax.tick_params(pad=5)
    ax.legend()

    axins = inset_axes(ax, width="30%", height="30%", loc="center", borderpad=2)
    axins.patch.set_facecolor("white")
    axins.set_zorder(2)
    for fp, df in data.items():
        x = df["epoch"].values[-5:]
        y = df[metric].values[-5:]
        axins.plot(x, y, linewidth=1.5, marker="o", markersize=3)

    x0 = min(df["epoch"].values[-5:].min() for df in data.values())
    x1 = max(df["epoch"].values[-5:].max() for df in data.values())
    y0 = min(df[metric].values[-5:].min() for df in data.values()) - 0.001
    y1 = max(df[metric].values[-5:].max() for df in data.values()) + 0.001

    axins.set_xlim(x0, x1)
    axins.set_ylim(y0, y1)
    axins.tick_params(labelsize=8)
    # axins.yaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5", linestyle="--", linewidth=0.5, zorder=1)
    plt.tight_layout()
    fig.savefig(os.path.join(research_dir, f"{metric}_{stage}_comparison.png"))
    
plot_loss()